### [andrewslotnick.com](http://andrewslotnick.com)
---
# Audio Delay with Python
[Notebook source code](https://github.com/aslotnick/audio_experiments/blob/master/Delay.ipynb)

_Delay_ is a fundamental audio effect. The idea of playing a sound and repeating it after some time is simple, but it is used extensively in music production as a standalone effect and as the basis of _reverb_, _chorus_, and _flanging_. My motivation for implementing this effect is to test out the  audio support in the standard library and to understand how delay works.

To implement a delay function we'll need to read and write audio data, add two audio signals together, change a signal's volume, and create empty space at the end. The final version of the function will be able to generate the following sound files (and many more) based on the input.

In [1]:
from IPython.display import Audio,display
print("Input:")
display(Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Trumpet.wav', embed=False))
print("Outputs with delay:")
display(Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Trumpet_delay_700ms.wav', embed=False),
        Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Trumpet_delay_1ms_0.75f_1n.wav', embed=False),
        Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Trumpet_delay_250ms_0.7f_10n.wav', embed=False))

Input:


Outputs with delay:


## Get some audio
The [Wave][] module handles input and output of [Wav][] audio files. Audio data is represented as a bytes object, which makes this standard library module less useful than some other modules that return a numpy array (e.g. [audiolab][]).

To work with the data in the bytes object we'll also need to know the number of [bytes per sample][] and the [sample frequency][]. These examples only support mono audio, so the nchannels parameter will need to be equal to 1.

[Wave]: https://docs.python.org/3.4/library/wave.html
[Wav]: https://en.wikipedia.org/wiki/WAV
[bytes per sample]: https://en.wikipedia.org/wiki/Audio_bit_depth
[audiolab]: https://pypi.python.org/pypi/scikits.audiolab
[sample frequency]: https://en.wikipedia.org/wiki/Sampling_(signal_processing)#Audio_sampling

In [2]:
import wave

def input_wave(filename,frames=10000000): #10000000 is an arbitrary large number of frames
    with wave.open(filename,'rb') as wave_file:
        params=wave_file.getparams()
        audio=wave_file.readframes(frames)  
        if params.nchannels!=1:
            raise Exception("The input audio should be mono for these examples")
    return params, audio

#output to file so we can use ipython notebook's Audio widget
def output_wave(audio, params, stem, suffix):
    #dynamically format the filename by passing in data
    filename=stem.replace('.wav','_{}.wav'.format(suffix))
    with wave.open(filename,'wb') as wave_file:
        wave_file.setparams(params)
        wave_file.writeframes(audio)

These short mono audio clips will demonstrate the delay effect. One is a trumpet solo and the other is the sound of a punch.

In [3]:
trumpet_params, trumpet_bytes = input_wave('wavs/Trumpet.wav')
punch_params,punch_bytes = input_wave('wavs/Punch.wav')

print("Bytes per sample: {}".format(trumpet_params.sampwidth), 
      "Samples per second: {}".format(trumpet_params.framerate),
      "First 10 bytes:", trumpet_bytes[:10], sep='\n')
display(Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Trumpet.wav', embed=False))

print("Bytes per sample: {}".format(punch_params.sampwidth), 
      "Samples per second: {}".format(punch_params.framerate),
      "First 10 bytes:", punch_bytes[:10], sep='\n')
display(Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Punch.wav', embed=False))

Bytes per sample: 3
Samples per second: 44100
First 10 bytes:
b'\x00\x00\x00\x13\x00\x00\x1f\x00\x005'


Bytes per sample: 2
Samples per second: 44100
First 10 bytes:
b'\x00\x00\x00\x00\x00\x00\x00\x00\xfe\xff'


## Implement a delay function
The simplest delay function will create a copy of the input, add some silence (0's in the bytes object) to the beginning of the copy, and combine it with the original input. The _add_ function from [Audioop](https://docs.python.org/3.4/library/audioop.html) will add the two bytes objects together. Audioop.add requires both pieces of audio to have the same length, so we also need to cut off the end of the copy. 


In [4]:
from audioop import add

def delay(audio_bytes,params,offset_ms):
    #calculate the number of bytes which corresponds to the offset in milliseconds
    offset= params.sampwidth*offset_ms*int(params.framerate/1000)
    #create some silence
    beginning= b'\0'*offset
    #remove space from the end
    end= audio_bytes[:-offset]
    return add(audio_bytes, beginning+end, params.sampwidth)

In [5]:
#1-second delay
delayed_bytes_1000=delay(trumpet_bytes,trumpet_params,1000)
output_wave(delayed_bytes_1000, trumpet_params, 'wavs/Trumpet.wav','delay_1000ms')
#250 ms delay
delayed_bytes_250=delay(punch_bytes,punch_params,250)
output_wave(delayed_bytes_250, punch_params, 'wavs/Punch.wav','delay_250ms')

display(
    Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Trumpet_delay_1000ms.wav', embed=False),
    Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Punch_delay_250ms.wav', embed=False)
    )

## Change the delayed audio's volume
To make this sound more like a realistic echo, we can change the volume of the delayed audio by multiplying it using audioop.mul. Note that multiplying each sample by one half is not the same as reducing the percieved loudness by one half.

In [6]:
from audioop import mul
#new delay function with factor
def delay(audio_bytes,params,offset_ms,factor=1):
    #calculate the number of bytes which corresponds to the offset in milliseconds
    offset= params.sampwidth*offset_ms*int(params.framerate/1000)
    #create some silence
    beginning= b'\0'*offset
    #remove space from the end
    end= audio_bytes[:-offset]
    #multiply by the factor
    multiplied_end= mul(audio_bytes[:-offset],params.sampwidth,factor)
    return add(audio_bytes, beginning+ multiplied_end, params.sampwidth)

In [7]:
#1-second delay with factor .5
delayed_bytes_1000=delay(trumpet_bytes,trumpet_params,offset_ms=1000, factor=0.5)
output_wave(delayed_bytes_1000, trumpet_params, 'wavs/Trumpet.wav','delay_1000ms_0.5f')
#500 ms delay with factor .25
delayed_bytes_250=delay(punch_bytes,punch_params,offset_ms=250, factor=0.25)
output_wave(delayed_bytes_250, punch_params, 'wavs/Punch.wav','delay_250ms_0.25f')

display(
    Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Trumpet_delay_1000ms_0.5f.wav', embed=False),
    Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Punch_delay_250ms_0.25f.wav', embed=False)
    )

## Multiple delays
Another enhancement to the delay function is to allow for multiple repeats. Each time a repeat occurs, the volume will get progressively louder or softer based on the factor.

In [8]:
from warnings import warn

def delay(audio_bytes,params,offset_ms,factor=1,num=1):
    if (factor>0.7 and num >3) or factor>1:
        warn("These settings may produce a very loud audio file. Please use caution when listening")
    #calculate the number of bytes which corresponds to the offset in milliseconds
    offset=params.sampwidth*offset_ms*int(params.framerate/1000)
    #add extra space at the end for the delays
    delayed_bytes=audio_bytes
    for i in range(num):
        #create some silence
        beginning = b'\0'*offset*(i+1)
        #remove space from the end
        end = audio_bytes[:-offset*(i+1)]
        #multiply by the factor
        multiplied_end= mul(end,params.sampwidth,factor**(i+1))
        delayed_bytes= add(delayed_bytes, beginning+multiplied_end, params.sampwidth)
    return delayed_bytes

In [9]:
#1-second delay with factor .5, 3 repeats
delayed_bytes_1000=delay(trumpet_bytes,trumpet_params,offset_ms=1000, factor=0.5, num=3)
output_wave(delayed_bytes_1000, trumpet_params, 'wavs/Trumpet.wav','delay_1000ms_0.5f_3n')
#250 ms delay with factor .7, 4 repeats
delayed_bytes_250=delay(punch_bytes, punch_params,offset_ms=250, factor=0.7, num=4)
output_wave(delayed_bytes_250, punch_params, 'wavs/Punch.wav','delay_250ms_0.7f_4n')

display(
    Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Trumpet_delay_1000ms_0.5f_3n.wav', embed=False),
    Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Punch_delay_250ms_0.7f_4n.wav', embed=False)
    )

## Leave space at the end
The multi-delays above output a sound of the same length as the original, so trying to add too many repeats will throw an exception. To give more time to hear the effects, let's increase the length to allow every repeat to finish.

In [10]:
def delay(audio_bytes,params,offset_ms,factor=1,num=1):
    if (factor>0.7 and num >3) or factor>1:
        warn("These settings may produce a very loud audio file. Please use caution when listening")
    #calculate the number of bytes which corresponds to the offset in milliseconds
    offset=params.sampwidth*offset_ms*int(params.framerate/1000)
    #add extra space at the end for the delays
    audio_bytes=audio_bytes+b'\0'*offset*(num)
    #create a copy of the original to apply the delays
    delayed_bytes=audio_bytes
    for i in range(num):
        #create some silence
        beginning = b'\0'*offset*(i+1)
        #remove space from the end
        end = audio_bytes[:-offset*(i+1)]
        #multiply by the factor
        multiplied_end= mul(end,params.sampwidth,factor**(i+1))
        delayed_bytes= add(delayed_bytes, beginning+multiplied_end, params.sampwidth)
    return delayed_bytes

In [11]:
#3-second delay with factor .5, 3 repeats
delayed_bytes_3000=delay(trumpet_bytes,trumpet_params,offset_ms=3000, factor=0.5, num=3)
output_wave(delayed_bytes_3000, trumpet_params, 'wavs/Trumpet.wav','delay_3000ms_0.5f_3n')
#500 ms delay with factor .7, 6 repeats
delayed_bytes_500=delay(punch_bytes, punch_params,offset_ms=500, factor=0.7, num=6)
output_wave(delayed_bytes_500, punch_params, 'wavs/Punch.wav','delay_500ms_0.7f_6n')

display(
    Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Trumpet_delay_3000ms_0.5f_3n.wav', embed=False),
    Audio(url='https://s3.amazonaws.com/audio-experiments/examples/Punch_delay_500ms_0.7f_6n.wav', embed=False)
    )

## Test out offset lengths and volume factors

The best way to understand how different parameters affect the final sound is to try out a lot of examples. delay_to_file is a helper function to speed up this process.

In [12]:
#helper function to try out lots of delays
def delay_to_file(audio_bytes,params,offset_ms,file_stem,factor=1,num=1):
    echoed_bytes=delay(audio_bytes,params,offset_ms,factor,num)
    output_wave(echoed_bytes, params, file_stem,'delay_{}ms_{}f_{}n'.format(offset_ms,factor,num))

A single delay of 1-2 seconds is too long to sound like a natural echo, so this can be used for a musical effect.

In [ ]:
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=2000,file_stem='wavs/Trumpet.wav',factor=.5)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=1000,file_stem='wavs/Trumpet.wav',factor=.5)

from IPython.display import display
display(
    Audio('wavs/Trumpet_delay_2000_0.5.wav'),
    Audio('wavs/Trumpet_delay_1000_0.5.wav')
    )

Adding 250-400 ms of delay gives the impression of a natural echo that might occur in a larger physical space.

In [ ]:
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=400, file_stem='wavs/Trumpet.wav',factor=.5)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=400, file_stem='wavs/Trumpet.wav',factor=.25)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=250, file_stem='wavs/Trumpet.wav',factor=.5)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=250, file_stem='wavs/Trumpet.wav',factor=.25)

display(
    Audio('wavs/Trumpet_delay_400_0.5.wav'),
    Audio('wavs/Trumpet_delay_400_0.25.wav'),
    Audio('wavs/Trumpet_delay_250_0.5.wav'),
    Audio('wavs/Trumpet_delay_250_0.25.wav')
    )

As delays get shorter, around 125 ms or less, the effect starts to be percieved as a single sound rather than a distinct echo. This produces the effect of a [comb filter](https://en.wikipedia.org/wiki/Comb_filter) as certain frequencies get louder or softer due to interference. On this trumpet sample, 1-5 ms delay sounds a lot like a trumpet played through a [mute](http://www.summersong.net/teacher/trumpetlessons/brassinstrumentmutes/).

In [ ]:
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=125,  file_stem='wavs/Trumpet.wav',factor=.5)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=75,  file_stem='wavs/Trumpet.wav',factor=.5)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=20,  file_stem='wavs/Trumpet.wav',factor=.75)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=5,  file_stem='wavs/Trumpet.wav',factor=.75)
delay_to_file(trumpet_bytes,trumpet_params, offset_ms=1,  file_stem='wavs/Trumpet.wav',factor=.75)

display(
    Audio('wavs/Trumpet_delay_125_0.5.wav'),
    Audio('wavs/Trumpet_delay_75_0.5.wav'),
    Audio('wavs/Trumpet_delay_20_0.75.wav'),
    Audio('wavs/Trumpet_delay_5_0.75.wav'),
    Audio('wavs/Trumpet_delay_1_0.75.wav')
    )       

In [ ]:
delay_to_file(trumpet_bytes,trumpet_params,offset_ms=50,file_stem='wavs/Trumpet.wav',factor=.76,num=10)
delay_to_file(trumpet_bytes,trumpet_params,offset_ms=250,file_stem='wavs/Trumpet.wav',factor=.7,num=10)
delay_to_file(trumpet_bytes,trumpet_params,offset_ms=1000,file_stem='wavs/Trumpet.wav',factor=.9,num=3)
display(
    Audio(filename='wavs/Trumpet_delay_50ms_0.76f_10n.wav'),
    Audio(filename='wavs/Trumpet_delay_250ms_0.7f_10n.wav'),
    Audio(filename='wavs/Trumpet_delay_1000ms_0.9f_3n.wav'),
)

In [ ]:
punch_params, punch_bytes = input_wave('wavs/Punch.wav') #must be mono
delayed_bytes_700=delay(punch_bytes,punch_params,offset_ms=50, factor=.75, num=2)
output_wave(delayed_bytes_700, punch_params, 'wavs/Punch.wav','delay')
Audio('wavs/Punch_delay.wav')